In [1]:
import sys
sys.executable

'/home/nithin/data2/nithin/anaconda3/bin/python'

In [2]:
import pandas as pd
import os

In [3]:
df = pd.read_csv('Train_ULAK.csv').dropna()
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,68855579,10,6,1038,11595,346,0,103.800000,167.133879,...,32,998.0000,0.0000,998,998,6.830000e+07,0.0000,68300000,68300000,DoS Hulk
1,53,196,2,2,70,174,35,35,35.000000,0.000000,...,32,0.0000,0.0000,0,0,0.000000e+00,0.0000,0,0,BENIGN
2,123,118,2,2,96,96,48,48,48.000000,0.000000,...,20,0.0000,0.0000,0,0,0.000000e+00,0.0000,0,0,BENIGN
3,80,295657,7,10,1114,15841,1084,0,159.142857,407.829796,...,20,0.0000,0.0000,0,0,0.000000e+00,0.0000,0,0,BENIGN
4,80,15705351,7,5,407,452,377,0,58.142857,140.620563,...,20,360718.0000,0.0000,360718,360718,9.767208e+06,0.0000,9767208,9767208,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783351,41272,98689782,8,5,11595,327,5792,0,1449.375000,2046.673464,...,32,13859.0000,0.0000,13859,13859,9.860000e+07,0.0000,98600000,98600000,BENIGN
1783352,443,90956783,63,77,1041,135313,373,0,16.523810,68.018939,...,32,198255.7778,362537.3486,1165022,77291,9.908053e+06,290822.6482,10000000,9132848,BENIGN
1783353,443,1241813,3,3,0,0,0,0,0.000000,0.000000,...,28,0.0000,0.0000,0,0,0.000000e+00,0.0000,0,0,BENIGN
1783354,53,71509,2,2,78,330,39,39,39.000000,0.000000,...,32,0.0000,0.0000,0,0,0.000000e+00,0.0000,0,0,BENIGN


In [4]:
!pip install scikit-learn

In [44]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
import torch

class RandomForestWrapper:
    def __init__(self, label_col, feature_cols=None, n_estimators=10, random_state=42, make_binary_on=None):
        self.label_col = label_col
        self.feature_cols = feature_cols
        self.n_estimators = n_estimators
        self.random_state = random_state
        self.model = RandomForestClassifier(n_estimators=self.n_estimators, random_state=self.random_state)
        self.imputer = SimpleImputer(strategy='mean')
        self.make_binary_on = make_binary_on
        
    def preprocess(self, df):
        # Replace inf and -inf with NaN
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        
        # Impute NaN values for numeric columns
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        self.imputer.fit(df[numeric_cols])
        df[numeric_cols] = self.imputer.transform(df[numeric_cols])
        
        # Check if all NaNs are filled
        if df.isnull().sum().sum() > 0:
            raise ValueError("DataFrame still contains NaN values after imputation.")

        
        # Extract feature and label data
        X = df.drop(columns=[self.label_col])
        y = df[self.label_col]
        
        # If feature_cols is specified, use only those columns
        if self.feature_cols:
            X = X[self.feature_cols]

        # If there is a label column on which to turn this binary
        if self.make_binary_on:
            print("Making binary on:", self.make_binary_on)
            y = df[' Label'] == self.make_binary_on
                
        return X, y
    
    def train(self, train_df):
        X_train, y_train = self.preprocess(train_df)
        self.model.fit(X_train, y_train)
    
    def test(self, test_df):
        X_test, y_test = self.preprocess(test_df)
        y_pred = self.model.predict(X_test)
        
        accuracy = accuracy_score(y_test, y_pred)
        
        return accuracy

In [48]:
class SpecializedRandomForestManager:
    def __init__(self):
        self.label_col = ' Label'
        # Create individual random forest classifiers for each label
        self.classifiers = {}
        
        # Define the feature columns for each attack category
        self.category_columns = {
            'BENIGN':[' Flow Duration', ' Fwd IAT Max'],
            'DoS Hulk':[' Fwd Packet Length Max', ' Flow Packets/s'],
            'DDoS':[' Flow Packets/s', ' Total Fwd Packets'],
            'PortScan': [' Destination Port', ' Fwd IAT Max'],
            'DoS GoldenEye': ['Fwd Packets/s', 'Bwd Packet Length Max'],
            'FTP-Patator': [' Fwd Packet Length Mean'],
            'DoS slowloris': [' Fwd IAT Max', ' Fwd IAT Mean'],
            'DoS Slowhttptest': [' Destination Port', ' Fwd IAT Mean'],
            'SSH-Patator': [' Fwd Packet Length Mean', ' Flow Packets/s'],
            'Web Attack – XSS': [' Fwd Packet Length Max'],
            'Web Attack – Brute Force': [' Fwd Packet Length Mean'],
            'Web Attack – Sql Injection': [' Fwd Packet Length Std']
        }
        
        # Initialize specialized classifiers
        for category, feature_cols in self.category_columns.items():
            self.classifiers[category] = RandomForestWrapper(label_col=self.label_col, feature_cols=feature_cols, make_binary_on=category)

    def train(self, df):
        for label, clf in self.classifiers.items():
            # Pre-filter the DataFrame based on the label and destination port conditions
            filtered_df = self.filter_df(df, label)

            print("Label:", label)
            # If the DataFrame is empty after filtering, skip this iteration
            if filtered_df.empty:
                continue
            print("filtered_df.shape", filtered_df.shape)
            
            # Train the classifier on the filtered DataFrame
            clf.train(filtered_df)

    def test(self, df):
        accuracies = {}
        for label, clf in self.classifiers.items():
            # Pre-filter the DataFrame based on the label and destination port conditions
            filtered_df = self.filter_df(df, label)

            print("Label:", label)
            # If the DataFrame is empty after filtering, skip this iteration
            if filtered_df.empty:
                continue

            print("filtered_df.shape", filtered_df.shape)

            # Test the classifier on the filtered DataFrame
            accuracies[label] = clf.test(filtered_df)

        return accuracies

    def filter_df(self, df, label):
        if label in ['BENIGN', 'Web Attack � Brute Force', 'Web Attack � Sql Injection', 'Web Attack � XSS']:
            return df[df[' Destination Port'].isin([80, 443])]
        elif label == 'SSH-Patator':
            return df[df[' Destination Port'] == 22]
        elif label == 'FTP-Patator':
            return df[df[' Destination Port'] == 21]

        return df


In [21]:
train_df = df
test_df = pd.read_csv('Test_ULAK.csv').dropna()
train_df, test_df

(          Destination Port   Flow Duration   Total Fwd Packets  \
 0                       80        68855579                  10   
 1                       53             196                   2   
 2                      123             118                   2   
 3                       80          295657                   7   
 4                       80        15705351                   7   
 ...                    ...             ...                 ...   
 1783351              41272        98689782                   8   
 1783352                443        90956783                  63   
 1783353                443         1241813                   3   
 1783354                 53           71509                   2   
 1783355               4306            2167                   2   
 
           Total Backward Packets  Total Length of Fwd Packets  \
 0                              6                         1038   
 1                              2                           70

In [49]:
# Initialize the manager class
manager = SpecializedRandomForestManager()

# Train the specialized classifiers
manager.train(train_df)

# Test the specialized classifiers
accuracies = manager.test(test_df)

# Print the accuracies
for label, acc in accuracies.items():
    print(f"Accuracy for {label}: {acc}")


Label: BENIGN
filtered_df.shape (708159, 79)


/tmp/ipykernel_937765/1052849751.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace([np.inf, -np.inf], np.nan, inplace=True)
/tmp/ipykernel_937765/1052849751.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_cols] = self.imputer.transform(df[numeric_cols])


Making binary on: BENIGN
Label: DoS Hulk
filtered_df.shape (1782497, 79)
Making binary on: DoS Hulk
Label: DDoS
filtered_df.shape (1782497, 79)
Making binary on: DDoS
Label: PortScan
filtered_df.shape (1782497, 79)
Making binary on: PortScan
Label: DoS GoldenEye
filtered_df.shape (1782497, 79)
Making binary on: DoS GoldenEye
Label: FTP-Patator
filtered_df.shape (8536, 79)
Making binary on: FTP-Patator
Label: DoS slowloris
filtered_df.shape (1782497, 79)


/tmp/ipykernel_937765/1052849751.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace([np.inf, -np.inf], np.nan, inplace=True)
/tmp/ipykernel_937765/1052849751.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_cols] = self.imputer.transform(df[numeric_cols])


Making binary on: DoS slowloris
Label: DoS Slowhttptest
filtered_df.shape (1782497, 79)
Making binary on: DoS Slowhttptest
Label: SSH-Patator
filtered_df.shape (10661, 79)
Making binary on: SSH-Patator
Label: Web Attack – XSS
filtered_df.shape (1782497, 79)


...

Making binary on: BENIGN
Label: DoS Hulk
filtered_df.shape (511828, 79)
Making binary on: DoS Hulk
Label: DDoS
filtered_df.shape (511828, 79)
Making binary on: DDoS
Label: PortScan
filtered_df.shape (511828, 79)
Making binary on: PortScan
Label: DoS GoldenEye
filtered_df.shape (511828, 79)
Making binary on: DoS GoldenEye
Label: FTP-Patator
filtered_df.shape (2439, 79)
Making binary on: FTP-Patator
Label: DoS slowloris
filtered_df.shape (511828, 79)


/tmp/ipykernel_937765/1052849751.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace([np.inf, -np.inf], np.nan, inplace=True)
/tmp/ipykernel_937765/1052849751.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_cols] = self.imputer.transform(df[numeric_cols])


Making binary on: DoS slowloris
Label: DoS Slowhttptest
filtered_df.shape (511828, 79)
Making binary on: DoS Slowhttptest
Label: SSH-Patator
filtered_df.shape (3106, 79)
Making binary on: SSH-Patator
Label: Web Attack – XSS
filtered_df.shape (511828, 79)


/tmp/ipykernel_937765/1052849751.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace([np.inf, -np.inf], np.nan, inplace=True)
/tmp/ipykernel_937765/1052849751.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_cols] = self.imputer.transform(df[numeric_cols])


Making binary on: Web Attack – XSS
Label: Web Attack – Brute Force
filtered_df.shape (511828, 79)
Making binary on: Web Attack – Brute Force
Label: Web Attack – Sql Injection
filtered_df.shape (511828, 79)
Making binary on: Web Attack – Sql Injection
Accuracy for BENIGN: 0.9532787167903995
Accuracy for DoS Hulk: 0.9780766194893598
Accuracy for DDoS: 0.9705233007963613
Accuracy for PortScan: 0.998054033776972
Accuracy for DoS GoldenEye: 0.9991325210813008
Accuracy for FTP-Patator: 0.980729807298073
Accuracy for DoS slowloris: 0.9993181303094009
Accuracy for DoS Slowhttptest: 0.9988531303484764
Accuracy for SSH-Patator: 0.9858338699291693
Accuracy for Web Attack – XSS: 1.0
Accuracy for Web Attack – Brute Force: 1.0
Accuracy for Web Attack – Sql Injection: 1.0
